# LLM Optimization Modelling Experiment

In [8]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [335]:
problem = '''We are looking at an alkylation process which will include the following 10 variables: olefin feed (barrels per day), isobutane recycle (barrels per day), acid addition rate (thousands of pounds per day), alkylate yield (barrels per day), isobutane makeup (barrels per day), acid strength (weight per cent), motor octane number, external isobutane-to-olefin ratio, acid dilution factor and F-4 performance number. 

We want to maximize the daily profit of this alkylation process. 
The profit is defined as the revenue generated from the alkylate yield multiplied with the motor octane number, minus the operational costs, which include olefin feed, isobutane recycle, acid addition rate, and isobutane makeup. 

Relationships in terms of other variables for alkylate yield, motor octane number, acid dilution factor, and F-4 performance number can be formulated as regression formulas. 
This regression estimate can deviate in both directions from true value of these variables by 2, 1, 5 and 10 percent, respectively.
Alkylate yield is a function of olefin feed and external isobutane-to-olefine yield. Alkalyte yield equals the amount of olefin feed multiplied by the sum of 1.12, 0.13167 times the external isobutane-to-olefin ratio and -0.00667 times the external isobutane-to-olefin ratio squared.
The motor octane number is derived from the external isobutane-to-olefin ratio and the acid strength. The motor octane number is calculated as the sum of 86.35, 1.098 time external isobutane-to-olefin ratio, -0.038 times the external isobutane-to-olefin ratio squared and 0.325 times acid strength reduced by 89.
The acid dilution factor is calculated based on the F-4 performance number. The acid dillution factor is expressed as 35.82 minus 0.222 times F-4 performance number.
Lastly, the F-4 performance number depends on the motor octane number. F-4 performance number is calculated as -133 plus three times the motor octane number.

There are some additional constraints imposed by the nature of the chemical process.
Each variable has a lower and an upper bound.
The external isobutane-to-olefin ratio needs to equal the ratio of isobutane recycle plus isobutane makeup to olefin feed. 
The acid strength needs to equal the ratio of 93000 times acid addition rate to acid addition rate multiplied by acid dilution factor in addition to 1000 times acid addition rate. 
Lastly, 1.22 alkylate yield needs to be equal to the combined olefin feed and isobutane makeup.'''

## 2. Ask for parameters

In [336]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate only the variables for this mathematical optimization problem. 
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [337]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Variables:

*  **OF:** Olefin feed (barrels per day)
*  **IBR:** Isobutane recycle (barrels per day)
*  **AAR:** Acid addition rate (thousands of pounds per day)
*  **AY**: Alkylate yield (barrels per day)
*  **IBM:** Isobutane makeup (barrels per day)
*  **AS**: Acid strength (weight percent)
*  **MON**: Motor octane number
*  **IOR:** External isobutane-to-olefin ratio
*  **ADF**: Acid dilution factor
*  **F4**: F-4 performance number 


# 2. Ask for objective

In [338]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please formulate only the objective function for this mathematical optimization problem."
prompt2 += problem + response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [339]:
Markdown(response2.text)

## Objective Function:

Maximize **Profit**

**Profit** = (**AY** * (1 + AY_deviation) * **MON** * (1 + MON_deviation) * Revenue_per_barrel) - (**OF** * Olefin_cost_per_barrel + **IBR** * Isobutane_cost_per_barrel + **AAR** * Acid_cost_per_thousand_pounds + **IBM** * Isobutane_cost_per_barrel)

Where:

* **Revenue_per_barrel:** Revenue generated per barrel of alkylate
* **Olefin_cost_per_barrel:** Cost per barrel of olefin feed
* **Isobutane_cost_per_barrel:** Cost per barrel of isobutane 
* **Acid_cost_per_thousand_pounds:** Cost per thousand pounds of acid
* **AY_deviation:** Deviation of the regression estimate for alkylate yield, within the range of -0.02 to +0.02
* **MON_deviation:** Deviation of the regression estimate for motor octane number, within the range of -0.01 to +0.01 

This objective function aims to find the values of the decision variables that maximize the profit, considering the revenue from alkylate yield influenced by its octane number and the costs associated with the input variables. 


# 3. Ask for constraints

In [340]:
#Second prompt gets the output of the previous step and generates the code
prompt3 = "Please formulate only the constraints for this mathematical optimization problem."
prompt3 += problem + response.text + response2.text
response3 = generative_multimodal_model.generate_content([prompt3])

In [341]:
Markdown(response3.text)

## Constraints:

**Regression Model Deviations:**

*  0.98 * **AY** <= **OF** * (1.12 + 0.13167 * **IOR** - 0.00667 * **IOR**^2) <= 1.02 * **AY** 
*  0.99 * **MON** <= 86.35 + 1.098 * **IOR** - 0.038 * **IOR**^2 + 0.325 * (**AS** - 89) <= 1.01 * **MON**
*  0.95 * **ADF** <= 35.82 - 0.222 * **F4** <= 1.05 * **ADF**
*  0.9 * **F4** <= -133 + 3 * **MON** <= 1.1 * **F4**

**Process Constraints:**

*  **IOR** = (**IBR** + **IBM**) / **OF**
*  **AS** = (93000 * **AAR**) / (**AAR** * **ADF** + 1000 * **AAR**) 
*  1.22 * **AY** = **OF** + **IBM**

**Variable Bounds:**

*  **OF_lower** <= **OF** <= **OF_upper**
*  **IBR_lower** <= **IBR** <= **IBR_upper**
*  **AAR_lower** <= **AAR** <= **AAR_upper**
*  **AY_lower** <= **AY** <= **AY_upper**
*  **IBM_lower** <= **IBM** <= **IBM_upper**
*  **AS_lower** <= **AS** <= **AS_upper**
*  **MON_lower** <= **MON** <= **MON_upper**
*  **IOR_lower** <= **IOR** <= **IOR_upper**
*  **ADF_lower** <= **ADF** <= **ADF_upper**
*  **F4_lower** <= **F4** <= **F4_upper**

Where "_lower" and "_upper" represent the lower and upper bounds for each variable, respectively. These bounds are determined by the operational limitations of the alkylation process. 


## 3. Generate the pyomo code

In [342]:
#Second prompt gets the output of the previous step and generates the code
prompt4 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt4 += response.text + response2.text + response3.text 
response4 = generative_multimodal_model.generate_content([prompt4])

In [343]:
## Showing the code in a formatted way
Markdown(response4.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data if available
Revenue_per_barrel = 60  
Olefin_cost_per_barrel = 20
Isobutane_cost_per_barrel = 15
Acid_cost_per_thousand_pounds = 50

OF_lower = 1000
OF_upper = 5000
IBR_lower =  500
IBR_upper =  2500
AAR_lower = 10
AAR_upper = 50
AY_lower =  2000
AY_upper = 10000
IBM_lower = 100
IBM_upper = 500
AS_lower = 89
AS_upper = 98
MON_lower = 89
MON_upper = 95
IOR_lower = 3
IOR_upper = 12
ADF_lower = 20
ADF_upper = 50
F4_lower = -150
F4_upper = -100

# Create a concrete model
model = pyo.ConcreteModel()

# Define decision variables
model.OF = pyo.Var(bounds=(OF_lower, OF_upper))
model.IBR = pyo.Var(bounds=(IBR_lower, IBR_upper))
model.AAR = pyo.Var(bounds=(AAR_lower, AAR_upper))
model.AY = pyo.Var(bounds=(AY_lower, AY_upper))
model.IBM = pyo.Var(bounds=(IBM_lower, IBM_upper))
model.AS = pyo.Var(bounds=(AS_lower, AS_upper))
model.MON = pyo.Var(bounds=(MON_lower, MON_upper))
model.IOR = pyo.Var(bounds=(IOR_lower, IOR_upper))
model.ADF = pyo.Var(bounds=(ADF_lower, ADF_upper))
model.F4 = pyo.Var(bounds=(F4_lower, F4_upper))

# Define deviations as variables 
model.AY_deviation = pyo.Var(bounds=(-0.02, 0.02))
model.MON_deviation = pyo.Var(bounds=(-0.01, 0.01))

# Define the objective function
def profit_rule(model):
    return (model.AY * (1 + model.AY_deviation) * model.MON * (1 + model.MON_deviation) * Revenue_per_barrel) - \
           (model.OF * Olefin_cost_per_barrel + model.IBR * Isobutane_cost_per_barrel + model.AAR * Acid_cost_per_thousand_pounds + \
            model.IBM * Isobutane_cost_per_barrel) 
model.profit = pyo.Objective(rule=profit_rule, sense=pyo.maximize)

# Define constraints
model.regression_AY = pyo.Constraint(expr= 0.98 * model.AY <= model.OF * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) )
model.regression_AY2 = pyo.Constraint(expr=model.OF * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2)  <= 1.02 * model.AY )
model.regression_MON = pyo.Constraint(expr= 0.99 * model.MON <= 86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * (model.AS - 89) )
model.regression_MON2 = pyo.Constraint(expr= 86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * (model.AS - 89) <= 1.01 * model.MON)
model.regression_ADF = pyo.Constraint(expr= 0.95 * model.ADF <= 35.82 - 0.222 * model.F4 )
model.regression_ADF2 = pyo.Constraint(expr= 35.82 - 0.222 * model.F4 <= 1.05 * model.ADF)
model.regression_F4 = pyo.Constraint(expr= 0.9 * model.F4  <= -133 + 3 * model.MON )
model.regression_F42 = pyo.Constraint(expr=-133 + 3 * model.MON <= 1.1 * model.F4)

model.process_IOR = pyo.Constraint(expr= model.IOR == (model.IBR + model.IBM) / model.OF)
model.process_AS = pyo.Constraint(expr= model.AS == (93000 * model.AAR) / (model.AAR * model.ADF + 1000 * model.AAR) )
model.process_AY = pyo.Constraint(expr= 1.22 * model.AY == model.OF + model.IBM)

# Solve the model
solver = pyo.SolverFactory('ipopt') # Choose a suitable solver, 'ipopt' is generally a good choice for nonlinear problems
results = solver.solve(model)

# Display the results
print(results)

# Print the optimal solution
print("Optimal Solution:")
print(f"OF (Olefin feed): {pyo.value(model.OF)} barrels per day")
print(f"IBR (Isobutane recycle): {pyo.value(model.IBR)} barrels per day")
print(f"AAR (Acid addition rate): {pyo.value(model.AAR)} thousand pounds per day")
print(f"AY (Alkylate yield): {pyo.value(model.AY)} barrels per day")
print(f"IBM (Isobutane makeup): {pyo.value(model.IBM)} barrels per day")
print(f"AS (Acid strength): {pyo.value(model.AS)} weight percent")
print(f"MON (Motor octane number): {pyo.value(model.MON)}")
print(f"IOR (External isobutane-to-olefin ratio): {pyo.value(model.IOR)}")
print(f"ADF (Acid dilution factor): {pyo.value(model.ADF)}")
print(f"F4 (F-4 performance number): {pyo.value(model.F4)}")
print(f"Maximum Profit: ${pyo.value(model.profit):.2f}")
```


## 4. Input problem data and try running the generated code

In [344]:
import pyomo.environ as pyo

# Sample Data - Replace with actual data if available
Revenue_per_barrel = 60  
Olefin_cost_per_barrel = 20
Isobutane_cost_per_barrel = 15
Acid_cost_per_thousand_pounds = 50

OF_lower = 1000
OF_upper = 5000
IBR_lower =  500
IBR_upper =  2500
AAR_lower = 10
AAR_upper = 50
AY_lower =  2000
AY_upper = 10000
IBM_lower = 100
IBM_upper = 500
AS_lower = 89
AS_upper = 98
MON_lower = 89
MON_upper = 95
IOR_lower = 3
IOR_upper = 12
ADF_lower = 20
ADF_upper = 50
F4_lower = -150
F4_upper = -100

# Create a concrete model
model = pyo.ConcreteModel()

# Define decision variables
model.OF = pyo.Var(bounds=(OF_lower, OF_upper))
model.IBR = pyo.Var(bounds=(IBR_lower, IBR_upper))
model.AAR = pyo.Var(bounds=(AAR_lower, AAR_upper))
model.AY = pyo.Var(bounds=(AY_lower, AY_upper))
model.IBM = pyo.Var(bounds=(IBM_lower, IBM_upper))
model.AS = pyo.Var(bounds=(AS_lower, AS_upper))
model.MON = pyo.Var(bounds=(MON_lower, MON_upper))
model.IOR = pyo.Var(bounds=(IOR_lower, IOR_upper))
model.ADF = pyo.Var(bounds=(ADF_lower, ADF_upper))
model.F4 = pyo.Var(bounds=(F4_lower, F4_upper))

# Define deviations as variables 
model.AY_deviation = pyo.Var(bounds=(-0.02, 0.02))
model.MON_deviation = pyo.Var(bounds=(-0.01, 0.01))

# Define the objective function
def profit_rule(model):
    return (model.AY * (1 + model.AY_deviation) * model.MON * (1 + model.MON_deviation) * Revenue_per_barrel) - \
           (model.OF * Olefin_cost_per_barrel + model.IBR * Isobutane_cost_per_barrel + model.AAR * Acid_cost_per_thousand_pounds + \
            model.IBM * Isobutane_cost_per_barrel) 
model.profit = pyo.Objective(rule=profit_rule, sense=pyo.maximize)

# Define constraints
model.regression_AY = pyo.Constraint(expr= 0.98 * model.AY <= model.OF * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2) )
model.regression_AY2 = pyo.Constraint(expr=model.OF * (1.12 + 0.13167 * model.IOR - 0.00667 * model.IOR**2)  <= 1.02 * model.AY )
model.regression_MON = pyo.Constraint(expr= 0.99 * model.MON <= 86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * (model.AS - 89) )
model.regression_MON2 = pyo.Constraint(expr= 86.35 + 1.098 * model.IOR - 0.038 * model.IOR**2 + 0.325 * (model.AS - 89) <= 1.01 * model.MON)
model.regression_ADF = pyo.Constraint(expr= 0.95 * model.ADF <= 35.82 - 0.222 * model.F4 )
model.regression_ADF2 = pyo.Constraint(expr= 35.82 - 0.222 * model.F4 <= 1.05 * model.ADF)
model.regression_F4 = pyo.Constraint(expr= 0.9 * model.F4  <= -133 + 3 * model.MON )
model.regression_F42 = pyo.Constraint(expr=-133 + 3 * model.MON <= 1.1 * model.F4)

model.process_IOR = pyo.Constraint(expr= model.IOR == (model.IBR + model.IBM) / model.OF)
model.process_AS = pyo.Constraint(expr= model.AS == (93000 * model.AAR) / (model.AAR * model.ADF + 1000 * model.AAR) )
model.process_AY = pyo.Constraint(expr= 1.22 * model.AY == model.OF + model.IBM)

# Solve the model
solver = pyo.SolverFactory('ipopt') # Choose a suitable solver, 'ipopt' is generally a good choice for nonlinear problems
results = solver.solve(model)

# Display the results
print(results)

# Print the optimal solution
print("Optimal Solution:")
print(f"OF (Olefin feed): {pyo.value(model.OF)} barrels per day")
print(f"IBR (Isobutane recycle): {pyo.value(model.IBR)} barrels per day")
print(f"AAR (Acid addition rate): {pyo.value(model.AAR)} thousand pounds per day")
print(f"AY (Alkylate yield): {pyo.value(model.AY)} barrels per day")
print(f"IBM (Isobutane makeup): {pyo.value(model.IBM)} barrels per day")
print(f"AS (Acid strength): {pyo.value(model.AS)} weight percent")
print(f"MON (Motor octane number): {pyo.value(model.MON)}")
print(f"IOR (External isobutane-to-olefin ratio): {pyo.value(model.IOR)}")
print(f"ADF (Acid dilution factor): {pyo.value(model.ADF)}")
print(f"F4 (F-4 performance number): {pyo.value(model.F4)}")
print(f"Maximum Profit: ${pyo.value(model.profit):.2f}")

model.name="unknown";
    - termination condition: infeasible
    - message from solver: Ipopt 3.11.1\x3a Converged to a locally infeasible
      point. Problem may be infeasible.

Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 12
  Sense: unknown
Solver: 
- Status: warning
  Message: Ipopt 3.11.1\x3a Converged to a locally infeasible point. Problem may be infeasible.
  Termination condition: infeasible
  Id: 200
  Error rc: 0
  Time: 0.10892200469970703
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal Solution:
OF (Olefin feed): 1402.0811168985301 barrels per day
IBR (Isobutane recycle): 2500.0 barrels per day
AAR (Acid addition rate): 10.121719837381065 thousand pounds per day
AY (Alkylate yield): 2000.0 barrels per day
IBM (Isobutane makeup): 500.0 barrels per day
AS (Acid strength): 89.0 weight percent
MON (Motor octane number): 89.0
IOR (External isobutane-to-olefin ratio)

## 5. Correct the code to verify model viability (optional)

## 6. Print the responses

In [345]:
print(response.text)

## Variables:

*  **OF:** Olefin feed (barrels per day)
*  **IBR:** Isobutane recycle (barrels per day)
*  **AAR:** Acid addition rate (thousands of pounds per day)
*  **AY**: Alkylate yield (barrels per day)
*  **IBM:** Isobutane makeup (barrels per day)
*  **AS**: Acid strength (weight percent)
*  **MON**: Motor octane number
*  **IOR:** External isobutane-to-olefin ratio
*  **ADF**: Acid dilution factor
*  **F4**: F-4 performance number 



In [346]:
print(response2.text)

## Objective Function:

Maximize **Profit**

**Profit** = (**AY** * (1 + AY_deviation) * **MON** * (1 + MON_deviation) * Revenue_per_barrel) - (**OF** * Olefin_cost_per_barrel + **IBR** * Isobutane_cost_per_barrel + **AAR** * Acid_cost_per_thousand_pounds + **IBM** * Isobutane_cost_per_barrel)

Where:

* **Revenue_per_barrel:** Revenue generated per barrel of alkylate
* **Olefin_cost_per_barrel:** Cost per barrel of olefin feed
* **Isobutane_cost_per_barrel:** Cost per barrel of isobutane 
* **Acid_cost_per_thousand_pounds:** Cost per thousand pounds of acid
* **AY_deviation:** Deviation of the regression estimate for alkylate yield, within the range of -0.02 to +0.02
* **MON_deviation:** Deviation of the regression estimate for motor octane number, within the range of -0.01 to +0.01 

This objective function aims to find the values of the decision variables that maximize the profit, considering the revenue from alkylate yield influenced by its octane number and the costs associated wi

In [347]:
print(response3.text)

## Constraints:

**Regression Model Deviations:**

*  0.98 * **AY** <= **OF** * (1.12 + 0.13167 * **IOR** - 0.00667 * **IOR**^2) <= 1.02 * **AY** 
*  0.99 * **MON** <= 86.35 + 1.098 * **IOR** - 0.038 * **IOR**^2 + 0.325 * (**AS** - 89) <= 1.01 * **MON**
*  0.95 * **ADF** <= 35.82 - 0.222 * **F4** <= 1.05 * **ADF**
*  0.9 * **F4** <= -133 + 3 * **MON** <= 1.1 * **F4**

**Process Constraints:**

*  **IOR** = (**IBR** + **IBM**) / **OF**
*  **AS** = (93000 * **AAR**) / (**AAR** * **ADF** + 1000 * **AAR**) 
*  1.22 * **AY** = **OF** + **IBM**

**Variable Bounds:**

*  **OF_lower** <= **OF** <= **OF_upper**
*  **IBR_lower** <= **IBR** <= **IBR_upper**
*  **AAR_lower** <= **AAR** <= **AAR_upper**
*  **AY_lower** <= **AY** <= **AY_upper**
*  **IBM_lower** <= **IBM** <= **IBM_upper**
*  **AS_lower** <= **AS** <= **AS_upper**
*  **MON_lower** <= **MON** <= **MON_upper**
*  **IOR_lower** <= **IOR** <= **IOR_upper**
*  **ADF_lower** <= **ADF** <= **ADF_upper**
*  **F4_lower** <= **F4** <= **F4_up

In [348]:
print(response4.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with actual data if available
Revenue_per_barrel = 60  
Olefin_cost_per_barrel = 20
Isobutane_cost_per_barrel = 15
Acid_cost_per_thousand_pounds = 50

OF_lower = 1000
OF_upper = 5000
IBR_lower =  500
IBR_upper =  2500
AAR_lower = 10
AAR_upper = 50
AY_lower =  2000
AY_upper = 10000
IBM_lower = 100
IBM_upper = 500
AS_lower = 89
AS_upper = 98
MON_lower = 89
MON_upper = 95
IOR_lower = 3
IOR_upper = 12
ADF_lower = 20
ADF_upper = 50
F4_lower = -150
F4_upper = -100

# Create a concrete model
model = pyo.ConcreteModel()

# Define decision variables
model.OF = pyo.Var(bounds=(OF_lower, OF_upper))
model.IBR = pyo.Var(bounds=(IBR_lower, IBR_upper))
model.AAR = pyo.Var(bounds=(AAR_lower, AAR_upper))
model.AY = pyo.Var(bounds=(AY_lower, AY_upper))
model.IBM = pyo.Var(bounds=(IBM_lower, IBM_upper))
model.AS = pyo.Var(bounds=(AS_lower, AS_upper))
model.MON = pyo.Var(bounds=(MON_lower, MON_upper))
model.IOR = pyo.Var(bounds=(IOR_lower, IOR